In [1]:
import os
import numpy as np
from pyrecode.recode_reader import ReCoDeReader
from collections import deque
from datetime import datetime
import time

%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import clear_output

In [5]:
import os
import numpy as np
from pyrecode.recode_reader import ReCoDeReader
from multiprocessing import Process, Manager
import time


class ReaderNode:

    def __init__(self, pid, filename, refresh_rate=1):

        self._pid = pid
        self._filename = filename
        self._refresh_rate = refresh_rate
        self._seek_table = []

        self._reader = None
        self._is_initialized = False
        self._node_states = None
        self._node_shapes = None
        self._node_fetch_params = None
        self._node_fetch_results = None
        self._ny = None
        self._nx = None

    def start(self, node_states, node_shapes, node_fetch_params, node_fetch_results):

        self._node_shapes = node_shapes
        self._node_states = node_states
        self._node_fetch_params = node_fetch_params
        self._node_fetch_results = node_fetch_results

        # wait for data and periodically check if its available
        self._node_states[self._pid] = 'init'
        while not self._is_initialized:
            time.sleep(self._refresh_rate)
            self._open()
            self._node_states[self._pid] = 'wait'

        while True:
            if self._node_states[self._pid] == 'close':
                self._close()
                break
            elif self._node_states[self._pid] == 'wait':
                time.sleep(self._refresh_rate)
            elif self._node_states[self._pid] == 'fetch':
                self._node_states[self._pid] = 'busy'
                d = self._get(self._node_fetch_params[self._pid]['from_frame'],
                              self._node_fetch_params[self._pid]['n_frames'])
                self._node_fetch_results[self._pid] = d
                self._node_states[self._pid] = 'wait'

    def _open(self):
        # check if file exists, and read only when available
        if os.path.isfile(self._filename):
            self._reader = ReCoDeReader(self._filename, is_intermediate=True)
            if not self._is_initialized:
                self._reader.open(print_header=False)
                header = self._reader.get_header().as_dict()
                self._ny = header['ny']
                self._nx = header['nx']
                self._node_shapes[self._pid] = (self._ny, self._nx)
                self._is_initialized = True

    def _get(self, from_frame, n_frames):

        if self._is_initialized:

            # get to the start of desired frame
            if len(self._seek_table) > from_frame:                
                self._reader._fp.seek(self._seek_table[from_frame])
            else:
                # loop through to frame_id. This is expensive and should be avoided
                self._reader.seek_to_frame_data()
                for i in range(from_frame):
                    f = self._reader.get_next_frame_raw(read_data=False)
                    if f is None:
                        break
                    else:
                        frame_id = list(f.keys())[0]
                        self._seek_table.append(f[frame_id]['data'])

            # get the next n_frames, updating _seek table in the process
            frames = {}
            for i in range(n_frames):
                f = self._reader.get_next_frame()
                if f is None:
                    break
                else:
                    frames.update(f)
                    if (i + from_frame) > len(self._seek_table):
                        p = self._reader.get_file_position()
                        self._seek_table.append(p)
            # print(self._seek_table)
            return frames
        else:
            return None

    def _close(self):
        self._reader.join()


class ReCoDeViewer:

    def __init__(self, folder_path, base_filename, num_parts, refresh_rate):

        self._folder_path = folder_path
        self._base_filename = base_filename

        self._num_parts = num_parts
        self._refresh_rate = refresh_rate

        self._node_states = None
        self._node_shapes = None
        self._node_fetch_params = None
        self._node_fetch_results = None
        self._readers = None

        self._ny = None
        self._nx = None

    def start(self):

        manager = Manager()
        self._node_states = manager.dict()
        self._node_shapes = manager.dict()
        self._node_fetch_params = manager.dict()
        self._node_fetch_results = manager.dict()

        self._readers = []
        for index in range(self._num_parts):
            inter_file_name = os.path.join(self._folder_path, self._base_filename + '_part' + '{0:03d}'.format(index))
            reader = ReaderNode(index, inter_file_name, self._refresh_rate)
            self._node_states[index] = 'init'
            self._node_shapes[index] = None
            self._node_fetch_params[index] = None
            self._node_fetch_results[index] = None
            reader_process = Process(target=reader.start, args=(self._node_states, self._node_shapes,
                                                                self._node_fetch_params, self._node_fetch_results))
            self._readers.append(reader_process)
            reader_process.start()

    def get_shape(self):
        if self._node_states[0] == 'wait':
            return self._node_shapes[0]
        else:
            return None

    def get(self, from_frame, n_frames):

        indicators = np.zeros(self._num_parts, dtype=np.bool)
        for index in range(self._num_parts):
            if self._node_states[index] == 'wait':
                n_frames_per_thread = int(np.ceil(n_frames/self._num_parts))
                self._node_fetch_params[index] = {'from_frame': from_frame, 'n_frames': n_frames_per_thread}
                self._node_states[index] = 'fetch'
            
        count = 0
        while count < self._num_parts:
            count = 0
            data = {}
            for index in range(self._num_parts):
                if self._node_states[index] == 'wait':
                    data.update(self._node_fetch_results[index])
                    indicators[index] = True
                    count += 1
                
        return data

    def close(self):
        count = 0
        for index in range(self._num_parts):
            if self._node_states[index] == 'wait':
                self._node_states[index] = 'close'
                count += 1

        if count == self._num_parts:
            return
        else:
            self.close()

In [8]:
_data_folder = '/scratch/loh/abhik/25_Aug_2020/live_view_test'
_dataset = ''
_tag = 'streampix_run_1'
_num_part_files = 10
_frames_per_fetch = 300
_frames_per_second = 60

viewer = ReCoDeViewer(os.path.join(_data_folder, _dataset), _tag + '.rc1', _num_part_files, 0.1)
viewer.start()

time_step = 0
time_point = 0

print("Waiting for reader to initialize...")
_shape = viewer.get_shape()
while _shape is None:
    _shape = viewer.get_shape() 
    time.sleep(0.1)
print("Reader ready, shape =", _shape)

frame_index = 0

while True:
    
    frames = viewer.get(frame_index, _frames_per_fetch)
    
    if frames is not None and len(frames) > 0:
        view = np.zeros(_shape, dtype=np.uint16)
        for frame_id in frames:
            frame_data = frames[frame_id]['data'].todense()
            view = np.add(view, frame_data)

        n_frames = len(frames)
        frame_ids = list(frames.keys())
        min_frame = np.min(frame_ids)
        max_frame = np.max(frame_ids)
        is_stale = ''
        frame_index += len(frames)
    else:
        is_stale = '\nWaiting for data...'
        
    checksum = np.sum(view)
    
    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(view)
    t = time_step * _frames_per_fetch/_frames_per_second
    title_str = 'Elapsed Time = ' + str(time_step) +  ' seconds'\
                '\nShowing Sum of ' + str(n_frames) + ' Frames starting from frame ' + str(frame_index) + \
                '\nchecksum = ' + str(checksum) + '\n' + is_stale
    ax.set(title=title_str)
    # fig.colorbar(im)
    plt.show()
    clear_output(wait=True)
    
    time_step += 1

Process Process-38:
Process Process-35:
Process Process-42:


KeyboardInterrupt: 

Process Process-43:
Process Process-44:
Process Process-40:
Process Process-39:
Process Process-41:
Traceback (most recent call last):
Process Process-36:
Process Process-37:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self